In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [2]:
print(path)

/kaggle/input/cifake-real-and-ai-generated-synthetic-images


In [3]:
import torch
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms

from transformers import ViTImageProcessor

model_name = "google/vit-large-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)

image_mean, image_std = processor.image_mean, processor.image_std

normalize = transforms.Normalize(mean=image_mean, std=image_std)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

CIFAKE_train_dataset = datasets.ImageFolder(root=f"{path}/train", transform=transform)
CIFAKE_test_dataset = datasets.ImageFolder(root=f"{path}/test", transform=transform)

val_ratio = 0.2

targets = CIFAKE_train_dataset.targets

train_indices, val_indices = train_test_split(
    list(range(len(targets))),
    test_size=val_ratio,
    stratify=targets,
    random_state=42
)

CIFAKE_train_dataset = Subset(CIFAKE_train_dataset, train_indices)
CIFAKE_val_dataset = Subset(CIFAKE_train_dataset.dataset, val_indices)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def collate_fn(examples):
    pixel_values, labels = zip(*examples)
    pixel_values = torch.stack(pixel_values)
    labels = torch.tensor(labels)
    return {"pixel_values": pixel_values, "labels": labels}

In [8]:
label2id = {'FAKE': 0, 'REAL': 1}
id2label = {0: 'FAKE', 1: 'REAL'}

In [9]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels = 2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    output_dir="output-models",
    num_train_epochs=5,
    eval_strategy="steps",  # (you had a typo: should be 'evaluation_strategy', not 'eval_strategy')
    eval_steps=1000,  # make sure this matches with logging_steps if you want them to align
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    logging_strategy="steps",
    logging_steps=1000,
    fp16=True,
    weight_decay=0.02,
    save_strategy="steps",
    save_steps=4000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",  # or another metric like "eval_loss", "f1", etc.
    greater_is_better=False,  # or False if lower is better (e.g., for loss)
    dataloader_pin_memory=False
)

trainer = Trainer(
    model,
    train_args,
    train_dataset=CIFAKE_train_dataset,
    eval_dataset=CIFAKE_val_dataset,
    data_collator=collate_fn,
    tokenizer=processor,
)

trainer.model.to(device)
trainer.train()

<ipython-input-15-df866642694c>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
1000,0.049000,0.127512
2000,0.047200,0.065889
3000,0.069200,0.070097
4000,0.056500,0.062536
5000,0.028000,0.059815
6000,0.028500,0.046559
7000,0.026100,0.044212
8000,0.021400,0.067577
9000,0.016200,0.038944
10000,0.012200,0.046793


TrainOutput(global_step=20000, training_loss=0.020312918335199357, metrics={'train_runtime': 4626.387, 'train_samples_per_second': 86.461, 'train_steps_per_second': 4.323, 'total_flos': 1.095736719458304e+20, 'train_loss': 0.020312918335199357, 'epoch': 5.0})

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
trainer.save_model("/content/drive/MyDrive/watchdog_ai")
processor.save_pretrained("/content/drive/MyDrive/watchdog_ai")

['/content/drive/MyDrive/watchdog_ai/preprocessor_config.json']

In [19]:
import shutil

# Replace these with your actual folder paths
source_folder = '/content/wandb'
destination_folder = '/content/drive/MyDrive/'

# Move the folder
shutil.move(source_folder, destination_folder)

'/content/drive/MyDrive/output-models'